In [31]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np 
import matplotlib.pyplot as plt 
torch.manual_seed(1)

In [12]:
#데이터 
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

W = torch.zeros((2,1), requires_grad = True) 
b = torch.zeros(1, requires_grad=True)

# hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) +b)))
hypothesis = torch.sigmoid(x_train.matmul(W)+b)

losses = -(y_train * torch.log(hypothesis) + (1-y_train) * torch.log(1-hypothesis)) #<--- lossfunction : binary cross entropy
cost = losses.mean()
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

In [24]:
import torch.nn as nn
class LogisticRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(2, 1)
    self.sigmoid = nn.Sigmoid()
  def forward(self, x):
    return self.sigmoid(self.linear(x))


In [ ]:
#모델 초기화 
# W = torch.zeros((2,1), requires_grad = True) 
# b = torch.zeros(1, requires_grad=True)

model = LogisticRegression()

#optimizer 설정 
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 10000 
for epoch in range(nb_epochs+1):

  hypothesis = model(x_train)
  cost = F.binary_cross_entropy(hypothesis, y_train)

  # cost로 H(x) 개선 
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
    correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
    accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
    print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
        epoch, nb_epochs, cost.item(), accuracy * 100,
    ))

# 다중 선형 회귀 

In [26]:
x_train = torch.FloatTensor([[73, 80, 75],
[93, 88, 93],
[89, 91, 90],
[96, 98, 100],
[73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [33]:
class MultiLinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3,1)

  def forward(self,x):
    return self.linear(x)

class CustomDataset(Dataset):
  def __init__(self,x_train,y_train):
    self.x_train = x_train
    self.y_train = y_train 
  
  def __len__(self):
    return len(self.x_train)

  def __getitem__(self,idx):
    x = torch.FloatTensor(self.x_train[idx])
    y = torch.FloatTensor(self.y_train[idx])
    return x, y 

In [ ]:
model = MultiLinearRegression()
optimizer = optim.Adam(model.parameters(),lr = 1e-5)
dataset = CustomDataset(x_train,y_train)
dataloader = DataLoader(dataset,batch_size=2,shuffle=True)

nb_epochs = 100 
for epoch in range(nb_epochs+1):
  for batch_idx, samples in enumerate(dataloader):
    batch_x, batch_y = samples 

    prediction = model(batch_x)

    cost = F.mse_loss(prediction,batch_y)
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

In [45]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(x_train,y_train)

LinearRegression()

In [52]:
np.mean((lr.predict(x_train) - np.array(y_train))**2)

0.1043246

array([[152.],
       [185.],
       [180.],
       [196.],
       [142.]], dtype=float32)